# Calculating ion solutions

## Configuration

Hydrogen only. Requires `dilute-lte` excitation and `nebular` ionization to provide the first approximation for the solver

In [1]:
from pathlib import Path

import tardis
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml((Path(tardis.__file__).parent / "plasma/tests/data/plasma_base_test_config.yml"))

config.model.abundances.He = 0
config.model.abundances.H = 1

config.plasma.excitation = "dilute-lte"
config.plasma.ionization = "nebular"

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

## Atomic data

Requires photoionization cross section data

In [2]:
from copy import deepcopy

from tardis.io.atom_data import AtomData
from tardis.model.base import SimulationState

atom_data = AtomData.from_hdf('/tardis-regression-data/atom_data/nlte_atom_data/TestNLTE_He_Ti.h5')

atom_data2 = deepcopy(atom_data)

# Prepare atom data for Hydrogen only, so we have access to the H photoionization data
atom_data2.prepare_atom_data([1], "macroatom", [], [(1, 0)])

sim_state = SimulationState.from_config(config, atom_data=atom_data)

Number of density points larger than number of shells. Assuming inner point irrelevant
model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!


In [3]:
import astropy.units as u
import numpy as np

from tardis.plasma.electron_energy_distribution import (
    ThermalElectronEnergyDistribution,
)
from tardis.plasma.radiation_field import (
    DilutePlanckianRadiationField,
)

# Test that the ionization calculations work for multiple cells and match the density config
rad_field = DilutePlanckianRadiationField(np.ones(20) * 10000 * u.K, dilution_factor=np.ones(20) * 1.0)

electron_dist = ThermalElectronEnergyDistribution(0, np.ones(20) * 10000 * u.K, np.ones(20) * 2e9 * u.cm**-3)

# Create a plasma

In [4]:
from tardis.plasma.assembly.base import PlasmaSolverFactory

plasma_solver_factory = PlasmaSolverFactory(atom_data, config)
plasma_solver_factory.prepare_factory([1],"tardis.plasma.properties.property_collections" )
plasma = plasma_solver_factory.assemble(sim_state.calculate_elemental_number_density(atom_data.atom_data.mass), rad_field, sim_state.time_explosion, electron_dist.number_density, link_t_rad_t_electron=1)

Zeta_data missing - replaced with 1s. Missing ions: []
Zeta_data missing - replaced with 1s. Missing ions: []


# Create LTE properties

In [5]:
from tardis.plasma.properties.ion_population import IonNumberDensity


class LTEIonNumberDensity(IonNumberDensity):
    outputs = ('lte_ion_number_density', )
    latex_name = ('N_{i,j}^*',)

    def calculate(self, phi_Te, lte_partition_function_Te, number_density, block_ids, ion_threshold):
        return self.calculate_with_n_electron(
            phi_Te, lte_partition_function_Te, number_density, self._electron_densities, block_ids, ion_threshold)

In [6]:
from tardis.plasma.properties.level_population import (
    LevelNumberDensity,
)


class LTELevelNumberDensity(LevelNumberDensity):
    outputs = ('lte_level_number_density',)
    latex_name = ('N_{i,j,k}^*',)

    def _calculate_dilute_lte(self, lte_level_boltzmann_factor_Te, lte_ion_number_density,
                              levels, lte_partition_function_Te):
        return super()._calculate_dilute_lte(
            lte_level_boltzmann_factor_Te, lte_ion_number_density, levels, lte_partition_function_Te)

In [7]:
lte_ion_number_density = LTEIonNumberDensity(plasma,
                                             electron_densities=plasma.electron_densities
                                             ).calculate(
                                                 plasma.thermal_phi_lte,
                                                 plasma.thermal_lte_partition_function,
                                                 plasma.number_density,
                                                 None, 1e-20)[0]

In [8]:
lte_level_number_density = LTELevelNumberDensity(plasma).calculate(
                                                     plasma.thermal_lte_level_boltzmann_factor,
                                                     lte_ion_number_density,
                                                     plasma.levels,
                                                     plasma.thermal_lte_partition_function)

# Initialize ionization solvers

In [9]:
from tardis.plasma.equilibrium.rate_matrix import IonRateMatrix
from tardis.plasma.equilibrium.rates import (
    AnalyticPhotoionizationRateSolver,
    CollisionalIonizationRateSolver,
)

photoionization_rate_solver = AnalyticPhotoionizationRateSolver(atom_data2.photoionization_data)

collisional_rate_solver = CollisionalIonizationRateSolver(atom_data2.photoionization_data)

ion_rate_matrix_solver = IonRateMatrix(photoionization_rate_solver, collisional_rate_solver)

## Optional: solve for radiative ionization rates

In [10]:
photoion_rates, recomb_rates = photoionization_rate_solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1],
    plasma.level_number_density.loc[plasma.level_number_density.index.get_level_values('ion_number') < 1],
    lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.ion_number_density.loc[plasma.ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.partition_function,
    plasma.general_level_boltzmann_factor,
    )

## Optional: solve for collisional ionization rates

In [11]:
level_to_ion_population_factor = lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1] / (
            lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1].values
            * electron_dist.number_density.value
        )

coll_ionization_rate, coll_recomb_rate = collisional_rate_solver.solve(
    electron_dist,
    level_to_ion_population_factor,
    plasma.partition_function,
    plasma.general_level_boltzmann_factor,
)

## Optional: solve for the ionization rate matrix

In [12]:
ion_rate_matrix = ion_rate_matrix_solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1],
    plasma.level_number_density.loc[plasma.level_number_density.index.get_level_values('ion_number') < 1],
    lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.ion_number_density.loc[plasma.ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.partition_function,
    plasma.general_level_boltzmann_factor,
    charge_conservation=True)

Ionization rate matrix for cell 0. First row is charge conservation. Second row is number conservation.

In [13]:
ion_rate_matrix[0].values

array([array([[ 0.00000000e+00,  1.00000000e+00, -1.00000000e+00],
              [-4.36681796e+02,  2.19607750e-03,  0.00000000e+00],
              [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00]])],
      dtype=object)

# Solve ion population

In [14]:
from tardis.plasma.equilibrium.ion_populations import IonPopulationSolver

solver = IonPopulationSolver(ion_rate_matrix_solver)

# Charge conservation is currently required to be False for the solver to work correctly.

ion_pops, electron_densities = solver.solve(
    rad_field,
    electron_dist,
    plasma.number_density,
    lte_level_number_density,
    plasma.level_number_density,
    lte_ion_number_density,
    plasma.ion_number_density,
    plasma.partition_function,
    plasma.general_level_boltzmann_factor,
    charge_conservation=False)

In [15]:
ion_pops

0             1             2  \
atomic_number ion_number                                             
1             0           2.639896e+05  1.998712e+05  1.527937e+05   
              1           4.506418e+10  3.422454e+10  2.626417e+10   

                                     3             4             5  \
atomic_number ion_number                                             
1             0           1.178332e+05  9.159744e+04  7.171794e+04   
              1           2.035073e+10  1.591079e+10  1.254396e+10   

                                     6             7             8  \
atomic_number ion_number                                             
1             0           5.652026e+04  4.480670e+04  3.571110e+04   
              1           9.967117e+09  7.977761e+09  6.429439e+09   

                                     9            10            11  \
atomic_number ion_number                                             
1             0           2.860052e+04  2.300787e+04  1.858514e+04   
              1           5.215159e+09  4.255999e+09  3.493221e+09   

                                    12            13            14  \
atomic_number ion_number                                             
1             0           1.507070e+04  1.226605e+04  1.001935e+04   
              1           2.882734e+09  2.391174e+09  1.993104e+09   

                                    15            16            17  \
atomic_number ion_number                                             
1             0           8.213483e+03  6.757461e+03  5.580090e+03   
              1           1.668991e+09  1.403731e+09  1.185571e+09   

                                    18            19  
atomic_number ion_number                              
1             0           4.625355e+03  3.848962e+03  
              1           1.005310e+09  8.556990e+08

In [16]:
lte_ion_number_density / lte_ion_number_density.sum()

0         1         2         3         4   \
atomic_number ion_number                                                     
1             0           0.000006  0.000006  0.000006  0.000006  0.000006   
              1           0.999994  0.999994  0.999994  0.999994  0.999994   

                                5         6         7         8         9   \
atomic_number ion_number                                                     
1             0           0.000006  0.000006  0.000006  0.000006  0.000006   
              1           0.999994  0.999994  0.999994  0.999994  0.999994   

                                10        11        12        13        14  \
atomic_number ion_number                                                     
1             0           0.000006  0.000006  0.000006  0.000006  0.000006   
              1           0.999994  0.999994  0.999994  0.999994  0.999994   

                                15        16        17        18        19  
atomic_number ion_number                                                    
1             0           0.000006  0.000006  0.000006  0.000006  0.000006  
              1           0.999994  0.999994  0.999994  0.999994  0.999994

In [17]:
electron_densities

0     4.506418e+10
1     3.422454e+10
2     2.626417e+10
3     2.035073e+10
4     1.591079e+10
5     1.254396e+10
6     9.967117e+09
7     7.977761e+09
8     6.429439e+09
9     5.215159e+09
10    4.255999e+09
11    3.493221e+09
12    2.882734e+09
13    2.391174e+09
14    1.993104e+09
15    1.668991e+09
16    1.403731e+09
17    1.185571e+09
18    1.005310e+09
19    8.556990e+08
dtype: float64